# Prostate Gland Extraction Together with Tumor Mask

You can extract one slice prostate gland per patient where tumor has the biggest form.

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
from ipywidgets import interactive, widgets
from IPython.display import display
from pathlib import Path

In [ ]:
def find_max_tumor_slice(mask_path):
    mask_img = nib.load(mask_path)
    mask_data = mask_img.get_fdata()
    tumor_slices = [(slice_number, mask_data[:, :, slice_number].sum()) for slice_number in range(mask_data.shape[2])]
    max_tumor_slice = max(tumor_slices, key=lambda x: x[1])
    return max_tumor_slice, mask_data

def get_tumor_coordinates(mask_data, threshold=0):
    tumor_coordinates = np.argwhere(mask_data > threshold)
    min_coords = np.min(tumor_coordinates, axis=0)
    max_coords = np.max(tumor_coordinates, axis=0)
    return min_coords, max_coords

def get_tumor_dimensions(min_coords, max_coords):
    width = max_coords[0] - min_coords[0] + 1
    height = max_coords[1] - min_coords[1] + 1
    depth = max_coords[2] - min_coords[2] + 1
    return width, height, depth

def extract_tumor_region(original_path, gland_mask_path, tumor_mask_path, output_path, patient_number, width, height, depth, dataset, width_slack=0, height_slack=0):
    if dataset == 'private':
        tumor_mask = nib.load(os.path.join(tumor_mask_path,f'{patient_number}_{patient_number}.nii.gz'))
        gland_mask = nib.load(os.path.join(gland_mask_path,f'{patient_number}_{patient_number}.nii.gz'))
    if dataset == 'picai':
        tumor_mask = nib.load(os.path.join(tumor_mask_path,f'{patient_number}.nii.gz'))
        gland_mask = nib.load(os.path.join(gland_mask_path,f'{patient_number}.nii.gz'))

    # Get data arrays
    tumor_mask_data = tumor_mask.get_fdata()

    # Get data arrays
    gland_mask_data = gland_mask.get_fdata()
    # Find the slice with the largest tumor region
    tumor_slices = np.sum(tumor_mask_data, axis=(0, 1))
    largest_slice = np.argmax(tumor_slices)

    # Find the center of the tumor region in the largest slice
    tumor_indices = np.argwhere(gland_mask_data[:, :, largest_slice] > 0)
    center = np.mean(tumor_indices, axis=0)
    nan_mask = np.isnan(center)
    if np.any(nan_mask):
        tumor_indices = np.argwhere(tumor_mask_data[:, :, largest_slice] > 0)
        center = np.mean(tumor_indices, axis=0)
    nan_mask = np.isnan(center)
    if np.any(nan_mask):
        print(patient_number)
        return

    start = max(0, (2*largest_slice - depth) // 2)
    end = min(gland_mask_data.shape[2], (2*largest_slice + depth) // 2)
    if (2*largest_slice - depth) // 2 < 0:
        start = 0
        end = depth
    
    if (2*largest_slice + depth) // 2 > gland_mask_data.shape[2]:
        start = gland_mask_data.shape[2] - depth
        end = gland_mask_data.shape[2]

    h_s = center[0] - (height / 2 + height_slack)
    h_e = center[0] + (height / 2 + height_slack)
    w_s = center[1] - (width / 2 + width_slack)
    w_e = center[1] + (width / 2 + width_slack)

    if center[0] - (height / 2 + height_slack) < 0:
        h_s = 0
        h_e = height + height_slack
    if center[0] + (height / 2 + height_slack) > gland_mask_data.shape[0]:
        h_s = gland_mask_data.shape[0] - (height + height_slack)
        h_e = gland_mask_data.shape[0]
    if center[1] - (width / 2 + width_slack) < 0:
        w_s = 0
        w_e = width + width_slack
    if center[1] + (width / 2 + width_slack) > gland_mask_data.shape[1]:
        w_s = gland_mask_data.shape[1] - (width + width_slack)
        w_e = gland_mask_data.shape[1]

    # Extract the mask region based on the same coordinates
    extracted_gland_mask = gland_mask_data[
        int(h_s):int(h_e),
        int(w_s):int(w_e),
        start:end
    ]
    # Extract the mask region based on the same coordinates
    extracted_tumor_mask = tumor_mask_data[
        int(h_s):int(h_e),
        int(w_s):int(w_e),
        start:end
    ]

    new_mask = nib.Nifti1Image(extracted_tumor_mask, tumor_mask.affine)
    if dataset == 'private':
        nib.save(new_mask, os.path.join(os.path.join(output_path, f'{patient_number}'),f'{patient_number}_{patient_number}.nii.gz'))
    if dataset == 'picai':
        nib.save(new_mask, os.path.join(os.path.join(output_path, f'{patient_number[:5]}'),f'{patient_number}.nii.gz'))

    
    files = os.listdir(os.path.join(original_path))

    # Filter files that start with "001"
    files = [file for file in files if file.startswith(patient_number)]

    for file in files:
        # Load NIfTI files
        img = nib.load(os.path.join(original_path,file))

        image_data = img.get_fdata()
        # Extract the image region based on the provided width, height, and depth with slack
        extracted_image = image_data[
            int(h_s):int(h_e),
            int(w_s):int(w_e),
            start:end
        ]
        # Set pixels in the original image to zero where the mask is black
        modified_data = np.where(extracted_gland_mask == 0, 0, extracted_image)

        # Save the extracted regions to new NIfTI files
        new_img = nib.Nifti1Image(modified_data, img.affine)
        if dataset == 'private':
            nib.save(new_img, os.path.join(os.path.join(output_path, f'{patient_number}'),file))
        if dataset == 'picai':
            nib.save(new_img, os.path.join(os.path.join(output_path, f'{patient_number[:5]}'),file))

In [ ]:
def extract_prostate(original_path, gland_mask_path, tumor_mask_path, output_path,dataset):
    global_max_tumor_region = None

    for mask_filename in os.listdir(gland_mask_path):
        if mask_filename.endswith(".nii.gz"):
            mask_file_path = os.path.join(gland_mask_path, mask_filename)
            max_tumor_slice, mask_data = find_max_tumor_slice(mask_file_path)

            if global_max_tumor_region is None or max_tumor_slice[1] > global_max_tumor_region[1]:
                global_max_tumor_region = (mask_filename, max_tumor_slice[0], max_tumor_slice[1], mask_data)

    # Get tumor coordinates in a bounding box
    min_coords, max_coords = get_tumor_coordinates(global_max_tumor_region[3])

    # Get tumor dimensions (width, height, depth)
    width, height, depth = get_tumor_dimensions(min_coords, max_coords)

    print(f"The global maximum tumor region is in {global_max_tumor_region[0]} on slice {global_max_tumor_region[1]} with a tumor pixel sum of {global_max_tumor_region[2]}")
    print(f"Tumor coordinates in bounding box: Min = {min_coords}, Max = {max_coords}")
    print(f"Tumor dimensions: Width = {width}, Height = {height}, Depth = {depth}")

    width = 120 
    height = 120
    depth = 1
    width_slack = 0
    height_slack = 0
    if dataset == 'private':
        directories = [d[:3] for d in os.listdir(tumor_mask_path) if d.endswith('nii.gz')]
    if dataset == 'picai':
        directories = [d[:-7] for d in os.listdir(tumor_mask_path) if d.endswith('nii.gz')]


    for patient_number in directories:
        if dataset == 'private':
            Path(os.path.join(output_path,f'{patient_number}')).mkdir(parents=True, exist_ok=True)
        if dataset == 'picai':
            Path(os.path.join(output_path,f'{patient_number[:5]}')).mkdir(parents=True, exist_ok=True)

        extract_tumor_region(original_path, gland_mask_path, tumor_mask_path, output_path, patient_number, width, height, depth, dataset, width_slack, height_slack)
        print(f"Extraction of Patient {patient_number} completed.")


In [ ]:
def convert_numpy_format(original_path, output_directory):
    
    # Get a list of all directories in the directory
    directories = [d for d in os.listdir(original_path) if os.path.isdir(os.path.join(original_path, d))]

    # Iterate through all NIfTI files in the input directory
    for patient_number in directories:
        # Create the output directory if it doesn't exist
        Path(os.path.join(output_directory,f'{patient_number}')).mkdir(parents=True, exist_ok=True)

        for file_name in os.listdir(os.path.join(original_path,f'{patient_number}')):

            if file_name.endswith('.nii.gz'):  # Ensure it's a NIfTI file
                input_path = os.path.join(os.path.join(original_path,f'{patient_number}'), file_name)
                # Load NIfTI image using nibabel
                nifti_img = nib.load(input_path)

                # Get the image data as a NumPy array
                image_data = nifti_img.get_fdata()
                name = os.path.splitext(os.path.splitext(file_name)[0])
                # Save the NumPy array to a file with the same name
                output_path = os.path.join(os.path.join(output_directory,f'{patient_number}'), f'{name[0]}.npy')
                np.save(output_path, image_data)

In [ ]:
original_path = '/local_ssd/practical_wise24/prostate_cancer/NNUNet_Lesion/Private_Dataset_Preprocessed_2/Private_Dataset/imagesTr'
gland_mask_path = "/local_ssd/practical_wise24/prostate_cancer/NNUNetModel/Gland_Segmentation/Results"
tumor_mask_path = "/local_ssd/practical_wise24/prostate_cancer/NNUNet_Lesion/Private_Dataset_Segmentation_Results"
output_path = '/local_ssd/practical_wise24/prostate_cancer/NNUNet_Lesion/Private_Extracted_1_Slice'
dataset = 'private'

print("Extraction Started")
extract_prostate(original_path, gland_mask_path, tumor_mask_path, output_path, dataset)
print("Extraction Completed")

In [ ]:
original_path = '/local_ssd/practical_wise24/prostate_cancer/NNUNet_Lesion/Private_Extracted_1_Slice'
output_path = '/local_ssd/practical_wise24/prostate_cancer/NNUNet_Lesion/Private_Extracted_1_Slice_Numpy'
convert_numpy_format(original_path, output_path)

In [ ]:
original_path = '/local_ssd/practical_wise24/prostate_cancer/NNUNet_Lesion/nnUNet_raw/Dataset600_Hum_AI/imagesTr'
gland_mask_path = "/local_ssd/practical_wise24/prostate_cancer/NNUNetModel/Gland_Segmentation/Results"
tumor_mask_path = "/local_ssd/practical_wise24/prostate_cancer/NNUNet_Lesion/nnUNet_raw/Dataset600_Hum_AI/labelsTr"
output_path = '/local_ssd/practical_wise24/prostate_cancer/NNUNet_Lesion/Picai_Extracted_1_Slice'
dataset = 'picai'

print("Extraction Started")
extract_prostate(original_path, gland_mask_path, tumor_mask_path, output_path, dataset)
print("Extraction Completed")

In [ ]:
original_path = '/local_ssd/practical_wise24/prostate_cancer/NNUNet_Lesion/Picai_Extracted_1_Slice'
output_path = '/local_ssd/practical_wise24/prostate_cancer/NNUNet_Lesion/Picai_Extracted_1_Slice_Numpy'
convert_numpy_format(original_path, output_path)